In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np


In [3]:
# load the pickle file
model = load_model('model.h5')

# load encoder and scaler
with open('OHE_geo.pkl','rb') as file:
    label_encoder_geo= pickle.load(file)

with open('label_encoder_gender.pkl','rb')as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)


In [17]:
# Example of i/p data
input_data ={
    'CreditScore' : 600,
    'Geography' : 'France',
    'Gender' : 'Male',
    'Age' : 40,
    'Tenure' : 3,
    'Balance' : 60000,
    'NumOfProducts' :2,
    'HasCrCard' : 1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [18]:
# Onehotencode '    ' 
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()

c:\Users\anike\anaconda3\envs\venv\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [19]:
geo_encoded_df = pd.DataFrame(geo_encoded,columns=label_encoder_geo.get_feature_names_out(['Geography']))

In [20]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [21]:
input_df = pd.DataFrame([input_data])

In [22]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [23]:
#Encode the categorical varible
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])

In [24]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [25]:
#concatination with OHE data
input_df = pd.concat([input_df.drop(columns=['Geography'],axis=1),geo_encoded_df],axis=1)

In [26]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [27]:
# Scaling the i/p data
input_scale = scaler.transform(input_df)

In [28]:
# Prediction 
y_pred = model.predict(input_scale)
y_pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step


array([[0.01212005]], dtype=float32)

In [30]:
pred_prb = y_pred[0][0]
pred_prb

np.float32(0.012120049)

In [31]:
if (pred_prb>0.5):
    print("Customer is likely to churn")

else:
    print('The customer is not likely to churn')

The customer is not likely to churn
